# --- Day 4: Giant Squid ---

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

![](../images/squid_0.png)

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

![](../images/squid_1.png)

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

![](../images/squid_2.png)


Finally, 24 is drawn:

![](../images/squid_3.png)

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: `14 21 17 24 4`).

**The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, `188 * 24 = 4512`.**

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?


In [1]:
from collections import defaultdict

In [2]:
import numpy as np

### Build necessary functions

In [3]:
def update_board(board, number):
    
    # Get board coordinates that match the number
    x_coord, y_coord = np.where(np.array(board) == number)
    
    # Update the board in place
    for x, y in zip(x_coord, y_coord):
        board[x][y] = True

def check_rows(board):

    for row in range(len(board)):
        if all(i == True for i in board[row]):
            return True
    return False

def check_cols(board):
    
    for col in range(len(board[0])):
        column = [row[col] for row in board]
        if all(i == True for i in column):
            return True
    return False

def get_board_value(board, num):
    
    remaining_board_val = 0
    
    for row_idx in range(len(board)):
        row = board[row_idx]
        for col_idx in range(len(row)):
            val = board[row_idx][col_idx]
            if val != True:
                remaining_board_val += val
    return remaining_board_val * num

def return_answer(test_numbers, boards):

    for num in test_numbers:
        for b_num in boards.keys():
            update_board(boards[b_num], num)

            if check_rows(boards[b_num]):
                return get_board_value(boards[b_num], num)

            if check_cols(boards[b_num]):
                return get_board_value(boards[b_num], num)


### Set up and run on the test input...

In [4]:
test_numbers = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]

boards = defaultdict(list)
boards[1] = [[22, 13, 17, 11, 0],
                     [8,  2, 23,  4, 24],
                     [21,  9, 14, 16,  7],
                     [6, 10,  3, 18,  5],
                     [1, 12, 20, 15, 19]]

boards[2] = [[3, 15, 0, 2, 22],
                     [9, 18, 13, 17,  5],
                     [19,  8, 7, 25, 23],
                     [20, 11, 10, 24,  4],
                     [14, 21, 16, 12,  6]]

boards[3] = [[14, 21, 17, 24,  4],
                     [10, 16, 15,  9, 19],
                     [18,  8, 23, 26, 20],
                     [22, 11, 13,  6,  5],
                     [2,  0, 12,  3,  7]]


known_answer = 4512

myanswer = return_answer(test_numbers, boards)

if myanswer == known_answer:
    print("Looks like its working!!")
else:
    print("There is something wrong... :(")

Looks like its working!!


### Try with the puzzle data

In [5]:
data_file = "../data/04_data.txt"

boards = defaultdict(list)

board_count = 0

with open(data_file, "r") as f:
    for idx, line in enumerate(f):
        
        if idx == 0:
            called_numers = [int(num) for num in line.split(",")]
        elif line == "\n":
            board_count += 1
        else:
            boards[board_count].append(
                [int(num.rstrip("\n"))
                 for num in line.split(" ")
                 if num is not ""]
            )

# Convert everything to an array for easy access
# for key in boards.keys():
#     boards[key] = np.array(boards[key])

In [6]:
return_answer(called_numers, boards)

22680

## --- Part Two ---

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?


In [7]:
def who_wins_last(test_numbers, boards):
    num_boards = len(boards.keys())
    board_keys = list(boards.keys())
    finished = []
    
    for num in test_numbers:
        for b_num in board_keys:
            update_board(boards[b_num], num)
            
            if b_num not in finished:

                if check_rows(boards[b_num]):
                    num_boards -= 1
                    finished.append(b_num)
                    if num_boards == 0:
                        return get_board_value(boards[b_num], num)

                if (b_num not in finished) and check_cols(boards[b_num]):
                    num_boards -= 1
                    finished.append(b_num)
                    if num_boards == 0:
                        return get_board_value(boards[b_num], num)

### Try it again on the test input

In [8]:
test_numbers = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]

boards = defaultdict(list)
boards[1] = [[22, 13, 17, 11, 0],
                     [8,  2, 23,  4, 24],
                     [21,  9, 14, 16,  7],
                     [6, 10,  3, 18,  5],
                     [1, 12, 20, 15, 19]]

boards[2] = [[3, 15, 0, 2, 22],
                     [9, 18, 13, 17,  5],
                     [19,  8, 7, 25, 23],
                     [20, 11, 10, 24,  4],
                     [14, 21, 16, 12,  6]]

boards[3] = [[14, 21, 17, 24,  4],
                     [10, 16, 15,  9, 19],
                     [18,  8, 23, 26, 20],
                     [22, 11, 13,  6,  5],
                     [2,  0, 12,  3,  7]]

known_answer = 1924

myanswer = who_wins_last(test_numbers, boards)

if myanswer == known_answer:
    print("Looks like its working!!")
else:
    print("There is something wrong... :(")

Looks like its working!!


### Now with the puzzle data...

In [9]:
data_file = "../data/04_data.txt"

boards = defaultdict(list)

board_count = 0

with open(data_file, "r") as f:
    for idx, line in enumerate(f):
        
        if idx == 0:
            called_numers = [int(num) for num in line.split(",")]
        elif line == "\n":
            board_count += 1
        else:
            boards[board_count].append(
                [int(num.rstrip("\n"))
                 for num in line.split(" ")
                 if num is not ""]
            )

who_wins_last(called_numers, boards)

16168